In [2]:
import os
import math

from functools import lru_cache
import pandas as pd
import numpy as np
from pypeg2 import *

import beoid.filename_parser

In [3]:
root = '/home/will/thesis/generated/beoid/test-1'
spatial_non_contrastive_pickle = os.path.join(root, 'spatial_non_contrastive_excitation_maps.pickle')
spatial_contrastive_pickle = os.path.join(root, 'spatial_contrastive_excitation_maps.pickle')
temporal_non_contrastive_pickle = os.path.join(root, 'temporal_non_contrastive_excitation_maps.pickle')
temporal_contrastive_pickle = os.path.join(root, 'temporal_contrastive_excitation_maps.pickle')

spatial_contrastive = pd.read_pickle(spatial_contrastive_pickle)
spatial_non_contrastive = pd.read_pickle(spatial_non_contrastive_pickle)
temporal_contrastive = pd.read_pickle(temporal_contrastive_pickle)
temporal_non_contrastive = pd.read_pickle(temporal_non_contrastive_pickle)


def clean_persisted_dataframe(df):
    """
    Instead of renaming all the columns of the data on disk and breaking all other code that uses it,
    we instead beautify it prior to using it for this analysis.
    """
    df.rename(columns={
        'attention_map': 'Attention Map',
        'frame': 'Frame',
        'starting_frame': 'Frame',
        'video': 'Clip',
        'type': 'Network',
    }, inplace=True)
    df['EBP Type'] = df['contrastive'].apply(lambda x: "Contrastive" if x else "Non-Contrastive")
    df.drop('contrastive', axis=1, inplace=True)
    capitalise = lambda s: s.title()
    df['Network'] = df['Network'].apply(capitalise)
    df['Window Size'] = df['Network'].apply(lambda net: 10 if net == "Temporal" else 1)
    return df
    
    
temporal_attention_maps = pd.concat([clean_persisted_dataframe(temporal_contrastive), 
                                     clean_persisted_dataframe(temporal_non_contrastive)])
spatial_attention_maps = pd.concat([clean_persisted_dataframe(spatial_contrastive), 
                                     clean_persisted_dataframe(spatial_non_contrastive)])
attention_maps = pd.concat([temporal_attention_maps, spatial_attention_maps])

def clip_info(row):
    clip = row[0]
    return (clip, int(parsed_clip.range.start), int(parsed_clip.range.end), str(parsed_clip.video))

def clip_offset_start(clip):
    return int(clip_range(clip).start)

def clip_offset_stop(clip):
    return int(clip_range(clip).end)
    
def clip_range(clip):
    return parse_clip(clip).range

def clip_video(clip):
    return compose(parse_clip(clip).video, autoblank=False)

def clip_operator(clip):
    return int(parse_clip(clip).video.subject)

def clip_location(clip):
    return parse_clip(clip).video.location.location

def clip_object(clip):
    return parse_clip(clip).objects[0]

def clip_action(clip):
    return parse_clip(clip).action

@lru_cache()
def parse_clip(clip):
    return parse(clip, beoid.filename_parser.Clip)
    

clips = attention_maps['Clip'].unique()
clip_info = pd.DataFrame({
    'Clip': clips,
    'Offset Start': list(map(clip_offset_start, clips)),
    'Offset Stop': list(map(clip_offset_stop, clips)),
    'Video': list(map(clip_video, clips)),
    
    'Operator': list(map(clip_operator, clips)),
    'Location': list(map(clip_location, clips)),
    'Object': list(map(clip_object, clips)),
    'Action': list(map(clip_action, clips)),
})
clip_info.set_index('Clip', inplace=True)

def get_clip_info(clip):
    return pd.Series([
        clip_video(clip),
        clip_operator(clip),
        clip_location(clip),
        clip_object(clip),
        clip_action(clip),
    ])
    

def video_frame(row):
    clip = row['Clip']
    offset_start = clip_info.loc[clip]['Offset Start']
    frame = row['Frame']
    return offset_start + frame

attention_maps['Video frame'] = attention_maps.apply(video_frame, axis=1)
attention_maps[['Video', 'Operator', 'Location', 'Object', 'Action']] = attention_maps['Clip'].apply(get_clip_info)
attention_maps.head()

,Attention Map,Frame,Network,Clip,EBP Type,Window Size,Video frame,Video,Operator,Location,Object,Action
0,"[[0.0, 0.0, 0.0, 0.0, 5.24706e-07, 1.75127e-06...",1,Temporal,04_Sink2_press_button_676-718,Contrastive,10,677,04_Sink2,4,Sink,button,press
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.88386e-07, 2...",2,Temporal,04_Sink2_press_button_676-718,Contrastive,10,678,04_Sink2,4,Sink,button,press
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.01044e-...",3,Temporal,04_Sink2_press_button_676-718,Contrastive,10,679,04_Sink2,4,Sink,button,press
3,"[[0.0, 0.0, 0.0, 0.0, 2.54073e-07, 9.79374e-07...",4,Temporal,04_Sink2_press_button_676-718,Contrastive,10,680,04_Sink2,4,Sink,button,press
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",5,Temporal,04_Sink2_press_button_676-718,Contrastive,10,681,04_Sink2,4,Sink,button,press


In [4]:
attention_maps.to_pickle('attention-maps-beoid-with-metadata.pickle')

In [5]:
clip_info.to_pickle('clip-info-beoid.pickle')